In [1]:
import gurobipy as gp
from gurobipy import GRB


In [2]:
import gurobipy as gp
from gurobipy import GRB

# Define the parameters and data
load_demand = [100, 120, 110, 130, 150]  # Load demand in kWh for each hour
pv_generation = [40, 50, 60, 70, 80]     # PV generation in kWh for each hour
battery_capacity = 200                   # Battery capacity in kWh
initial_battery_charge = 100             # Initial battery charge in kWh
battery_charge_rate = 0.5                # Battery charge rate (fraction of battery capacity)
battery_discharge_rate = 0.7             # Battery discharge rate (fraction of battery capacity)
grid_buy_price = [0.15, 0.18, 0.20, 0.17, 0.16]  # Grid buy price in $/kWh for each hour
grid_sell_price = [0.17, 0.12, 0.11, 0.10, 0.09]  # Grid sell price in $/kWh for each hour

# Create a Gurobi model
model = gp.Model("Microgrid_Optimization")

# Create decision variables
battery_charge = model.addVars(len(load_demand), lb=0, ub=battery_capacity, name="BatteryCharge")
grid_import = model.addVars(len(load_demand), lb=0, name="GridImport")
grid_export = model.addVars(len(load_demand), lb=0, name="GridExport")

# Define the objective function: minimize the total cost
total_cost = gp.quicksum(grid_buy_price[t] * grid_import[t] - grid_sell_price[t] * grid_export[t]
                         for t in range(len(load_demand)))
model.setObjective(total_cost, GRB.MINIMIZE)

# Add constraints
for t in range(len(load_demand)):
    model.addConstr(battery_charge[t] == battery_charge_rate * battery_capacity * (grid_import[t] - grid_export[t]))
    model.addConstr(battery_charge[t] <= battery_capacity)  # Battery capacity constraint
    model.addConstr(battery_charge[t] >= 0)                # Non-negativity constraint
    model.addConstr(grid_import[t] >= load_demand[t] - pv_generation[t] - battery_discharge_rate * battery_capacity)
    model.addConstr(grid_import[t] <= load_demand[t] - pv_generation[t] + (1 - battery_discharge_rate) * battery_capacity)
    model.addConstr(grid_export[t] <= pv_generation[t])     # Grid export constraint

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for t in range(len(load_demand)):
        print(f"Hour {t+1}: Battery Charge = {battery_charge[t].X:.2f} kWh, Grid Import = {grid_import[t].X:.2f} kWh, Grid Export = {grid_export[t].X:.2f} kWh")
    print(f"Total Cost: ${model.ObjVal:.2f}")
else:
    print("Optimization was not successful.")



Using license file C:\Users\moise\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 30 rows, 15 columns and 40 nonzeros
Model fingerprint: 0x4fa192a2
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [9e-02, 2e-01]
  Bounds range     [2e+02, 2e+02]
  RHS range        [4e+01, 2e+02]
Presolve removed 30 rows and 15 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -8.0000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds
Optimal objective -8.000000000e-01
Optimal solution found:
Hour 1: Battery Charge = 0.00 kWh, Grid Import = 40.00 kWh, Grid Export = 40.00 kWh
Hour 2: Battery Charge = 0.00 kWh, Grid Import = 0.00 kWh, Grid Export = 0.00 kWh
Hour 3: Battery Charge = 0.00 kWh, Grid Import = 0.00 kWh, Grid Export = 0.00 kWh


In [3]:
# Create a new model
model = gp.Model("Microgrid Optimization")

# Parameters
pv_capacity = 100  # PV capacity in kW
battery_capacity = 200  # Battery capacity in kWh
charger_power = 50  # Charger power in kW
demand = [80, 100, 120, 90, 110]  # Hourly demand in kW

# Variables
pv_generation = model.addVars(5, lb=0, name="PV_Generation")
battery_charge = model.addVars(5, lb=0, ub=battery_capacity, name="Battery_Charge")
battery_discharge = model.addVars(5, lb=0, ub=battery_capacity, name="Battery_Discharge")
charger_usage = model.addVars(5, lb=0, name="Charger_Usage")
grid_import = model.addVars(5, lb=0, name="Grid_Import")

# Objective
total_cost = pv_capacity * 1000 + battery_capacity * 200 + charger_power * 300
for i in range(5):
    total_cost += grid_import[i] * 0.1  # Simplified cost per kWh
model.setObjective(total_cost, sense=GRB.MINIMIZE)

# Constraints
for i in range(5):
    model.addConstr(pv_generation[i] <= pv_capacity)
    model.addConstr(battery_charge[i] <= battery_capacity)
    model.addConstr(battery_discharge[i] <= battery_capacity)
    model.addConstr(charger_usage[i] <= charger_power)
    model.addConstr(pv_generation[i] + grid_import[i] == demand[i] + battery_discharge[i] + charger_usage[i])

# Solve the optimization problem
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for i in range(5):
        print(f"Hour {i+1}: PV Generation = {pv_generation[i].x:.2f} kW, "
              f"Battery Charge = {battery_charge[i].x:.2f} kWh, "
              f"Charger Usage = {charger_usage[i].x:.2f} kW, "
              f"Grid Import = {grid_import[i].x:.2f} kWh")
    print("Total Cost:", model.objVal)
else:
    print("Optimization problem was not solved to optimality")

# Dispose of the model
model.dispose()


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 25 rows, 25 columns and 40 nonzeros
Model fingerprint: 0x2ea5ca75
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e-01]
  Bounds range     [2e+02, 2e+02]
  RHS range        [5e+01, 2e+02]
Presolve removed 25 rows and 25 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5500300e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  1.550030000e+05
Optimal solution found
Hour 1: PV Generation = 80.00 kW, Battery Charge = 0.00 kWh, Charger Usage = 0.00 kW, Grid Import = 0.00 kWh
Hour 2: PV Generation = 100.00 kW, Battery Charge = 0.00 kWh, Charger Usage = 0.00 kW, Grid Import = 0.00 kWh
Hour 3: PV Generation = 100.00 kW, Battery Charge = 0.00 kWh, Charger Usage

In [4]:
# Create a new Gurobi model
model = gp.Model("Microgrid_Optimization")

# Parameters
battery_capacity = 1000  # kWh
pv_capacity = 500  # kW
charging_power = 150  # kW (per charging station)
load_demand = 800  # kW
num_charging_stations = 4

# Variables
battery_energy = model.addVar(lb=0, ub=battery_capacity, name="BatteryEnergy")
pv_generation = model.addVar(lb=0, ub=pv_capacity, name="PVGeneration")
charging_power_vars = [model.addVar(lb=0, ub=charging_power, name=f"ChargingPower_{i}") for i in range(num_charging_stations)]

# Objective: Maximize PV generation and minimize grid power consumption
model.setObjective(pv_generation - gp.quicksum(charging_power_vars), GRB.MAXIMIZE)

# Constraints
# Energy balance equation
model.addConstr(pv_generation + battery_energy >= load_demand + gp.quicksum(charging_power_vars), name="EnergyBalance")

# Battery state of charge constraint
model.addConstr(battery_energy <= battery_capacity, name="BatterySOC")

# Optimize the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found")
    print(f"PV Generation: {pv_generation.X:.2f} kW")
    print(f"Battery Energy: {battery_energy.X:.2f} kWh")
    for i, charging_power_var in enumerate(charging_power_vars):
        print(f"Charging Power at Station {i + 1}: {charging_power_var.X:.2f} kW")
    print(f"Total Grid Power Consumption: {load_demand + sum(charging_power_var.X for charging_power_var in charging_power_vars):.2f} kW")
else:
    print("No feasible solution found")

# Dispose of the model
model.dispose()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 6 columns and 7 nonzeros
Model fingerprint: 0x3a6ff52e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+02, 1e+03]
  RHS range        [8e+02, 1e+03]
Presolve removed 2 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  5.000000000e+02
Optimal solution found
PV Generation: 500.00 kW
Battery Energy: 300.00 kWh
Charging Power at Station 1: 0.00 kW
Charging Power at Station 2: 0.00 kW
Charging Power at Station 3: 0.00 kW
Charging Power at Station 4: 0.00 kW
Total Grid Power Consumption: 800.00 kW


In [5]:
import gurobipy as gp
from gurobipy import GRB

# Create a Gurobi model
model = gp.Model("MicroGridOptimization")

# Parameters
# Define your parameters such as energy production, demand, and grid cost here.
# Example:
# - solar_generation: solar panel energy generation
# - battery_capacity: maximum battery capacity
# - grid_cost: cost of purchasing energy from the main grid
# - demand: energy demand of loads

solar_generation = [100, 150, 120, 80]  # Example solar generation values
battery_capacity = 200  # Example battery capacity
grid_cost = 0.1  # Example grid cost per unit of energy
demand = [90, 110, 100, 70]  # Example energy demand values

# Decision variables
# Binary variable representing whether to buy energy from the main grid or not
buy_from_grid = model.addVars(len(demand), vtype=GRB.BINARY, name="buy_from_grid")

# Continuous variable representing how much energy to draw from the grid
grid_energy = model.addVars(len(demand), lb=0, name="grid_energy")

# Continuous variable representing how much energy to use from the battery
battery_energy = model.addVars(len(demand), lb=0, ub=battery_capacity, name="battery_energy")

# Objective function: Minimize the cost of buying energy from the main grid
model.setObjective(gp.quicksum(grid_cost * grid_energy[i] for i in range(len(demand))), GRB.MINIMIZE)

# Constraints
for i in range(len(demand)):
    # Energy balance constraint
    model.addConstr(
        solar_generation[i] + battery_energy[i] + grid_energy[i] >= demand[i],
        f"Energy_Balance_{i}"
    )

    # Use battery energy first before buying from the grid
    model.addConstr(
        battery_energy[i] >= (demand[i] - solar_generation[i]) * (1 - buy_from_grid[i]),
        f"Battery_Use_{i}"
    )

# Solve the optimization problem
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    print("Optimal decision variables:")
    for i in range(len(demand)):
        print(f"Load {i+1}: Buy from grid = {buy_from_grid[i].x}, Grid energy = {grid_energy[i].x}, Battery energy = {battery_energy[i].x}")
    print(f"Total cost: {model.ObjVal}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 12 columns and 16 nonzeros
Model fingerprint: 0x1c4e2134
Variable types: 8 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e-01, 1e-01]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+01, 4e+01]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Optimal solution found.
Optimal decision variables:
Load 1: Buy from grid = -0.0, Grid energy = 0.0, Battery energy = 0.0
Load 2: Buy from grid = -0.0, Grid energy = 0.0, Battery energy = 0.0
Load 3: Buy from grid = -0.0, Grid energy = 0.0, Battery energy = 0.0
Load 4: Buy fr